<a href="https://colab.research.google.com/github/danielsoy/ADer/blob/main/trae_effi_b5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import time

Parameters that need to be defined before the choice

In [ ]:
IMG_SIZE = 224
EPOCHS = 30
SAVE_PATH = ''
DATA_PATH = ''
BATCH_SIZE = 16

Set environment (1: Local CPU, 2: Colab GPU)

In [ ]:
choice = "2"  # Hardcoded choice for local GPU training

In [ ]:
if choice == "2":
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_PATH = '/content/drive/MyDrive/good'
    SAVE_PATH = '/content/drive/MyDrive'
else:
    DATA_PATH = 'c:\\Users\\libro\\OneDrive\\Escritorio\\treads\\good'
    SAVE_PATH = 'c:\\Users\\libro\\OneDrive\\Escritorio\\treads'
    BATCH_SIZE = 8  # Smaller batch size for CPU
    tf.config.set_soft_device_placement(True)

In [ ]:
def create_model():
    # Base EfficientNetB5 model
    base_model = EfficientNetB5(
        include_top=False,
        weights='imagenet',
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    # Freeze base model
    base_model.trainable = False

    # Create encoder-decoder architecture
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model(inputs)

    # Encoder additional layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    encoded = layers.Dense(512, activation='relu', name='encoder_output')(x)

    # Decoder layers
    x = layers.Dense(1024, activation='relu')(encoded)
    x = layers.Dense(IMG_SIZE * IMG_SIZE * 3, activation='sigmoid')(x)
    outputs = layers.Reshape((IMG_SIZE, IMG_SIZE, 3))(x)

    return Model(inputs, outputs)

Load and preprocess data

In [ ]:
def load_data():
    images = []
    for filename in os.listdir(DATA_PATH):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(DATA_PATH, filename)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img.astype('float32') / 255.
            images.append(img)
    return np.array(images)

Set up GPU

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Load data

In [ ]:
print("Loading data...")
x_train = load_data()
print(f"Loaded {len(x_train)} images")

Create and compile model

In [ ]:
model = create_model()
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

Train model<br>
Add callbacks

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(
        os.path.join(SAVE_PATH, 'checkpoints/model_{epoch:02d}.keras'),
        save_best_only=True,
        monitor='val_loss'
    ),
    TensorBoard(log_dir=os.path.join(SAVE_PATH, 'logs/fit/' + time.strftime('%Y%m%d-%H%M%S')))
]

Modified training section

In [ ]:
history = model.fit(
    x_train,
    x_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
    shuffle=True,
    callbacks=callbacks,
    verbose=1
)

Add training visualization

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

In [ ]:
plt.tight_layout()
plt.savefig(os.path.join(SAVE_PATH, 'training_metrics.png'))
plt.close()

Save model and history<br>
Modified save paths

In [ ]:
model.save(os.path.join(SAVE_PATH, 'efficientnet_anomaly_model.keras'))
np.save(os.path.join(SAVE_PATH, 'training_history.npy'), history.history)

In [ ]:
print("Training completed and model saved!")